In [1]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np


Using TensorFlow backend.


In [2]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'data/en_bn.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [3]:
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


Number of samples: 523
Number of unique input tokens: 56
Number of unique output tokens: 61
Max sequence length for inputs: 10
Max sequence length for outputs: 26


In [4]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.


In [5]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('logs/testbn.h5')

Train on 418 samples, validate on 105 samples
Epoch 1/100
418/418 [==============================] - 2s 4ms/step - loss: 1.7566 - val_loss: 1.6527
Epoch 2/100
418/418 [==============================] - 1s 2ms/step - loss: 1.4854 - val_loss: 1.6056
Epoch 3/100
418/418 [==============================] - 1s 2ms/step - loss: 1.4301 - val_loss: 1.5792
Epoch 4/100
418/418 [==============================] - 1s 2ms/step - loss: 1.3984 - val_loss: 1.5880
Epoch 5/100
418/418 [==============================] - 1s 2ms/step - loss: 1.3716 - val_loss: 1.5441
Epoch 6/100
418/418 [==============================] - 1s 2ms/step - loss: 1.3509 - val_loss: 1.5457
Epoch 7/100
418/418 [==============================] - 1s 2ms/step - loss: 1.3374 - val_loss: 1.5232
Epoch 8/100
418/418 [==============================] - 1s 2ms/step - loss: 1.3019 - val_loss: 1.5426
Epoch 9/100
418/418 [==============================] - 1s 2ms/step - loss: 1.2917 - val_loss: 1.5103
Epoch 10/100
418/418 [=======================

Epoch 81/100
418/418 [==============================] - 1s 2ms/step - loss: 0.1464 - val_loss: 1.6594
Epoch 82/100
418/418 [==============================] - 1s 2ms/step - loss: 0.1271 - val_loss: 1.7018
Epoch 83/100
418/418 [==============================] - 1s 2ms/step - loss: 0.1462 - val_loss: 1.6566
Epoch 84/100
418/418 [==============================] - 1s 2ms/step - loss: 0.1364 - val_loss: 1.6913
Epoch 85/100
418/418 [==============================] - 1s 2ms/step - loss: 0.1194 - val_loss: 1.7109
Epoch 86/100
418/418 [==============================] - 1s 2ms/step - loss: 0.1165 - val_loss: 1.6836
Epoch 87/100
418/418 [==============================] - 1s 2ms/step - loss: 0.1258 - val_loss: 1.8413
Epoch 88/100
418/418 [==============================] - 1s 2ms/step - loss: 0.1052 - val_loss: 1.7469
Epoch 89/100
418/418 [==============================] - 1s 2ms/step - loss: 0.1162 - val_loss: 1.7377
Epoch 90/100
418/418 [==============================] - 1s 2ms/step - loss: 0.0948

/home/semanticslab13/anaconda3/envs/tf3/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [6]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


In [7]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [8]:
# #Testing with a single line

# input_sentence = "How are you?"
# test_sentence_tokenized = np.zeros(
#   (1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
# for t, char in enumerate(input_sentence):
#   test_sentence_tokenized[0, t, input_token_index[char]] = 1.
# print(input_sentence)
# print(decode_sequence(test_sentence_tokenized))

In [10]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)


-
Input sentence: Go.
Decoded sentence: যান।

-
Input sentence: Hi.
Decoded sentence: হাই।

-
Input sentence: Run!
Decoded sentence: চালান!

-
Input sentence: Run!
Decoded sentence: চালান!

-
Input sentence: Wow!
Decoded sentence: কি দারুন!

-
Input sentence: Fire!
Decoded sentence: ফায়ার!

-
Input sentence: Help!
Decoded sentence: সাহায্য করুন!

-
Input sentence: Jump.
Decoded sentence: আমা দেরেছি।

-
Input sentence: Stop!
Decoded sentence: বন্ধ করুন!

-
Input sentence: Stop!
Decoded sentence: বন্ধ করুন!

-
Input sentence: Stop!
Decoded sentence: বন্ধ করুন!

-
Input sentence: Wait!
Decoded sentence: অপেক্ষা কর

-
Input sentence: Wait!
Decoded sentence: অপেক্ষা কর

-
Input sentence: Go on.
Decoded sentence: যান।

-
Input sentence: Go on.
Decoded sentence: যান।

-
Input sentence: Go on.
Decoded sentence: যান।

-
Input sentence: Hello!
Decoded sentence: হ্যালো!

-
Input sentence: Hello!
Decoded sentence: হ্যালো!

-
Input sentence: I see.
Decoded sentence: আমি এটি ব্যবহার করি.

-
Input s